## 1. Знакомимся с данными

Задание 1.1
Сколько различных полных названий команд в таблице teams?
select
    count(distinct long_name)
from sql.teams

Сколько в таблице teams команд с коротким названием VAL?
select
    count(short_name)
from sql.teams
where short_name like 'VAL%'

Информацию о скольких матчах содержит таблица matches?
select
    count(*)
from sql.matches

Данные за какие сезоны даны в таблице matches?
Ответ введите в формате 2019/2020.
select
    distinct season
from sql.matches
order by season desc

Задание 1.2
Напишите запрос, который выведет сезон (season), общее количество забитых мячей домашними (total_home_goals) и гостевыми (total_away_goals) командами.
Отсортируйте по столбцу с сезоном в порядке возрастания.
select
    season,
    sum (home_team_goals) as total_home_goals,
    sum (away_team_goals) as total_away_goals
from sql.matches
group by season
order by season asc

## 2. Соединение таблиц по ключу

 декартовым произведением таблиц.
SELECT *
FROM
    sql.teams,
    sql.matches

Ключ — это поле (столбец) в таблице, которое позволяет однозначно идентифицировать запись (строку).

SELECT *
FROM
    sql.teams,
    sql.matches
WHERE home_team_api_id = api_id

Ключи бывают двух основных типов:
Primary — первичный ключ — служит для идентификации текущей таблицы и, как правило, идёт первым в списке столбцов. Всегда уникален: повторяющихся значений в основной таблице быть не может.
Foreign — внешний ключ — представляет собой ссылку на другую таблицу.

SELECT 
    long_name, 
    home_team_goals,
    away_team_goals
FROM
    sql.teams,
    sql.matches
WHERE home_team_api_id = api_id


Задание 2.2
Напишите запрос, который выведет таблицу с результатами матчей away, содержащую:

названия гостевых команд (long_name);
количество забитых мячей домашней команды (home_team_goals);
количество забитых мячей гостевой команды (away_team_goals).

SELECT 
    long_name, 
    home_team_goals,
    away_team_goals
FROM
    sql.teams,
    sql.matches
where away_team_api_id = api_id

## 3. Знакомимся с JOIN

Синтаксис:

SELECT
        столбец1,
	столбец2,
	...
FROM
	таблица1
JOIN таблица2 ON условие
JOIN таблица3 ON условие

Пример:
SELECT 
    long_name,
    home_team_goals,
    away_team_goals
FROM    
    sql.teams
JOIN sql.matches on home_team_api_id = api_id

Задание 3.2
Напишите запрос, который выведет два столбца: id матча (match_id) и id домашней команды (team_id), — а затем отсортируйте по id матча в порядке возрастания значений.

SELECT
    matches.id as match_id,
    teams.id as team_id
FROM 
    sql.teams
join sql.matches on home_team_api_id = api_id
order by matches.id asc

SELECT
    h.long_name "домашняя команда",
    m.home_team_goals "голы домашней команды",
    m.away_team_goals "голы гостевой команды",
    a.long_name "гостевая команда" 
FROM
    sql.matches m
    JOIN sql.teams h ON m.home_team_api_id = h.api_id
    JOIN sql.teams a ON m.away_team_api_id = a.api_id

Задание 3.3
Напишите запрос, который выведет столбцы: id матча, короткое название домашней команды (home_short), короткое название гостевой команды (away_short).
Отсортируйте запрос по возрастанию id матча.

SELECT 
    m.id,
    h.short_name "домашняя команда",
    a.short_name "гостевая команда" 
FROM
    sql.matches m
    JOIN sql.teams h ON m.home_team_api_id = h.api_id
    JOIN sql.teams a ON m.away_team_api_id = a.api_id
order by id asc

## 4. Фильтрация и агрегатные функции

SELECT 
    m.id
FROM
    sql.teams t
    JOIN sql.matches m ON m.away_team_api_id = t.api_id
WHERE long_name = 'Arsenal'

Задание 4.1
Напишите запрос, который выведет полное название команды (long_name), количество голов домашней команды (home_goal) и количество голов гостевой команды (away_goal) в матчах, где домашней командой были команды с коротким названием ‘GEN’.
Отсортируйте запрос по id матча в порядке возрастания.

SELECT 
    t.long_name,
    sum(m.home_team_goals) as home_goal,
    sum(m.away_team_goals) as away_goal
FROM
    sql.teams t
    JOIN sql.matches m ON m.home_team_api_id = t.api_id
WHERE t.short_name = 'GEN'
group by t.long_name, m.id
order by m.id asc

Задание 4.2
Напишите запрос, чтобы вывести id матчей, короткое название домашней команды (home_short), короткое название гостевой команды (away_short) для матчей сезона 2011/2012, в которых участвовала команда с названием Liverpool.
Отсортируйте по id матча в порядке возрастания.

SELECT
    m.id,
    t.short_name home_short,
    t1.short_name away_short
FROM
    sql.matches m
JOIN sql.teams t ON m.home_team_api_id = t.api_id
JOIN sql.teams t1 ON m.away_team_api_id = t1.api_id
WHERE
    m.season = '2011/2012'
    and (t.long_name = 'Liverpool' or t1.long_name = 'Liverpool')
ORDER BY m.id

АГРЕГАЦИЯ ДАННЫХ
Например, мы можем вывести сумму голов по командам для матчей, где команда выступала в гостях.

SELECT
    t.long_name,
    SUM(m.home_team_goals) + SUM(m.away_team_goals) match_goals
FROM
    sql.matches m
    JOIN sql.teams t ON m.away_team_api_id = t.api_id 
GROUP BY t.id

Поставим задачу — вывести таблицу с суммарным количеством забитых голов в матчах по командам и сезонам для команд, в которых суммарное количество голов в матчах сезона больше 100.

SELECT
    m.season,
    t.long_name,
    SUM(m.home_team_goals) + SUM(m.away_team_goals) total_goals
FROM sql.matches m
JOIN sql.teams t ON t.api_id = m.home_team_api_id OR t.api_id = m.away_team_api_id
GROUP BY m.season, t.id
HAVING SUM(m.home_team_goals) + SUM(m.away_team_goals) > 100

Задание 4.3
Напишите запрос, с помощью которого можно вывести список полных названий команд, сыгравших в гостях 150 и более матчей.
Отсортируйте список по названию команды.

SELECT 
    t.long_name
FROM
    sql.teams t
    JOIN sql.matches m ON m.away_team_api_id = t.api_id
group by t.id
HAVING count(m.id) >= 150
order by t.long_name

## 5. Способы соединения таблиц

Для INNER JOIN работает следующее правило: присоединяются только те строки таблиц, которые удовлетворяют условию соединения. Если в любой из соединяемых таблиц находятся такие строки, которые не удовлетворяют заявленному условию, — они отбрасываются.

SELECT 
COUNT(DISTINCT t.id) 
FROM 
sql.teams t
JOIN sql.matches m ON t.api_id = m.home_team_api_id OR t.api_id = m.away_team_api_id


Для LEFT JOIN работает следующее правило: из левой (относительно оператора) таблицы сохраняются все строки, а из правой добавляются только те, которые соответствуют условию соединения. Если в правой таблице не находится соответствия, то значения строк второй таблицы будут иметь значение NULL.

SELECT
    t.long_name,
    m.id
FROM sql.teams t
LEFT JOIN sql.matches m ON t.api_id = m.home_team_api_id OR t.api_id = m.away_team_api_id
ORDER BY m.id DESC
--WHERE m.id IS NULL

Ещё пример с агрегатными функциями:
SELECT
    t.long_name,
    SUM(m.away_team_goals) total_goals
FROM   
    sql.teams t
LEFT JOIN sql.matches m ON t.api_id = m.away_team_api_id
GROUP BY t.id
ORDER BY 2 DESC


Задание 5.1
--Используя LEFT JOIN, выведите список уникальных названий команд, содержащихся в таблице matches. Отсортируйте список в алфавитном порядке.

SELECT
  DISTINCT t.long_name
FROM
  sql.teams t
LEFT JOIN sql.matches m ON t.api_id = m.home_team_api_id or t.api_id = m.away_team_api_id
WHERE m.id is not null
ORDER BY 1

Задание 5.2
Используя LEFT JOIN, напишите запрос, который выведет полное название команды (long_name), количество матчей, в которых участвовала команда, — домашних и гостевых (matches_cnt). Отсортируйте по количеству матчей в порядке возрастания.

SELECT
    t.long_name,
    COUNT(m.id)
FROM 
sql.teams t
LEFT JOIN sql.matches m ON t.api_id = m.home_team_api_id or t.api_id = m.away_team_api_id
GROUP BY t.id
ORDER BY 2

Оператор FULL OUTER JOIN объединяет в себе LEFT и RIGHT JOIN и позволяет сохранить кортежи обеих таблиц. Даже если не будет соответствий, мы сохраним все записи из обеих таблиц.

SELECT 
…
FROM
	table1
FULL OUTER JOIN table2 ON условие

CROSS JOIN соединяет таблицы так, что каждая запись в первой таблице присоединяется к каждой записи во второй таблице, иначе говоря, даёт декартово произведение.

SELECT *
FROM
    sql.teams
    --JOIN sql.matches
    JOIN sql.matches ON TRUE

Пример:
SELECT
    DISTINCT
    t1.long_name home_team, 
    t2.long_name away_team
FROM
    sql.teams t1
    CROSS JOIN sql.teams t2

Задание 5.3
Напишите запрос, который выведет все возможные уникальные комбинации коротких названий домашней команды (home_team) и коротких названий гостевой команды (away_team).
Отсортируйте запрос по первому и второму столбцам.

select
    distinct
    t1.short_name home_team,
    t2.short_name away_team

from sql.teams t1,
    sql.teams t2
    
order by t1.short_name, t2.short_name

## 6. Итоги. Закрепление знаний

Задание 6.1
Напишите запрос, который выведет список уникальных полных названий команд (long_name), игравших в гостях в матчах сезона 2012/2013.
Отсортируйте список в алфавитном порядке.

select
    distinct t.long_name
from sql.teams t
    join sql.matches m on t.api_id = m.away_team_api_id
where season = '2012/2013'
order by 1

Задание 6.2
Напишите запрос, который выведет полное название команды (long_name) и общее количество матчей (matches_cnt), сыгранных командой Inter в домашних матчах.

select
    t.long_name,
    count(t.id) matches_cnt
from sql.teams t
    join sql.matches m on t.api_id = m.home_team_api_id
where long_name = 'Inter'
group by 1

Задание 6.3
Напишите запрос, который выведет ТОП-10 команд (long_name) по суммарному количеству забитых голов в гостевых матчах. Во втором столбце запроса выведите суммарное количество голов в гостевых матчах (total_goals).

select 
    t.long_name,
    sum(m.away_team_goals) total_goals
from sql.teams t
    join sql.matches m on t.api_id = m.away_team_api_id
group by 1
order by 2 desc
limit 10

Задание 6.4
Выведите количество матчей между командами Real Madrid CF и FC Barcelona.

select 
    count(m.id) matches_cnt
from sql.matches m
    left join sql.teams t1 on m.home_team_api_id = t1.api_id or m.away_team_api_id = t1.api_id
    left join sql.teams t2 on m.home_team_api_id = t2.api_id or m.away_team_api_id = t2.api_id
where t1.long_name = 'Real Madrid CF' and t2.long_name = 'FC Barcelona'

Задание 6.5
Напишите запрос, который выведет название команды (long_name), сезон (season) и суммарное количество забитых голов в домашних матчах (total_goals).
Оставьте только те строки, в которых суммарное количество голов менее десяти.
Отсортируйте запрос по названию команды, а затем — по сезону.

select
    t.long_name,
    m.season,
    sum(m.home_team_goals) total_goals
from sql.teams t
    left join sql.matches m on t.api_id = m.home_team_api_id
group by 1, 2
having sum(m.home_team_goals)<10
order by 1, 2